# LSTM

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
import plotly.graph_objects as go
import plotly.express as px
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import load_model
import yfinance as yf

In [2]:
data = yf.download("BNB-USD", start ="2017-11-09", end = "2023-05-22")

[*********************100%***********************]  1 of 1 completed


In [3]:
data.to_csv("dataset.csv")

In [4]:
data = pd.read_csv("dataset.csv")

In [5]:
data = pd.read_csv("dataset.csv")
panjang_data = len(data)* 0.8
print("Panjang data:", panjang_data)

Panjang data: 1616.0


In [6]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-11-09,2.053140,2.174230,1.893940,1.990770,1.990770,19192200
1,2017-11-10,2.007730,2.069470,1.644780,1.796840,1.796840,11155000
2,2017-11-11,1.786280,1.917750,1.614290,1.670470,1.670470,8178150
3,2017-11-12,1.668890,1.672800,1.462560,1.519690,1.519690,15298700
4,2017-11-13,1.526010,1.735020,1.517600,1.686620,1.686620,12238800
...,...,...,...,...,...,...,...
2015,2023-05-17,311.369812,314.269989,307.641205,314.083527,314.083527,437721579
2016,2023-05-18,314.091217,314.666290,306.649628,309.456696,309.456696,383049498
2017,2023-05-19,309.446777,311.197601,307.930054,308.964294,308.964294,334636708
2018,2023-05-20,308.982422,311.842316,308.087982,310.719971,310.719971,277393160


In [7]:
fig = px.line(data, x="Date", y="Adj Close")
fig.show()

In [8]:
data = data[['Date', 'Adj Close']]

In [9]:
data['Date'] = pd.to_datetime(data['Date'])

<ipython-input-9-1a72a8b5b870>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
data.set_index('Date', inplace=True)

In [11]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [12]:
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

In [14]:
#best_lstm_model = load_model('/content/model_lstm_bestparameter.h5')

In [15]:
# Create sequences for LSTM
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 30
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

Jika memanggil model berhenti running sampai sini

In [ ]:
# Function to create LSTM model
def create_lstm_model(units, learning_rate):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=units))
    model.add(Dense(units=1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'units': [50, 100],
    'learning_rate': [0.1, 0.01, 0.001],
    'epochs': [50, 100],
    'batch_size': [32, 64]
}

In [ ]:
# Create the LSTM model
lstm_model = KerasRegressor(build_fn=create_lstm_model, verbose=0)

<ipython-input-9-67c370388704>:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  lstm_model = KerasRegressor(build_fn=create_lstm_model, verbose=0)


In [ ]:
# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=lstm_model, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f4bae27e2c0>,
             param_grid={'batch_size': [32, 64], 'epochs': [50, 100],
                         'learning_rate': [0.1, 0.01, 0.001],
                         'units': [50, 100]})

In [ ]:
# Get the best hyperparameters
best_params_lstm = grid_search.best_params_
print("Best Hyperparameters (LSTM):", best_params_lstm)

In [ ]:
# Train the LSTM model with the best hyperparameters
best_lstm_model = create_lstm_model(units=best_params_lstm['units'], learning_rate=best_params_lstm['learning_rate'])
best_lstm_model.fit(X_train, y_train, epochs=best_params_lstm['epochs'], batch_size=best_params_lstm['batch_size'])

In [ ]:
#best_lstm_model.save('model_lstm_bestparameter.h5')

Running lagi mulai dari bawah ini

In [16]:
train_predictions_lstm_tuned = best_lstm_model.predict(X_train)
test_predictions_lstm_tuned = best_lstm_model.predict(X_test)

12/12 [==============================] - 0s 3ms/step


In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate RMSE
test_rmse_lstm_tuned = np.sqrt(mean_squared_error(y_test, test_predictions_lstm_tuned))

# Calculate MSE
test_mse_lstm_tuned = mean_squared_error(y_test, test_predictions_lstm_tuned)

# Calculate MAPE
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

test_mape_lstm_tuned = calculate_mape(y_test, test_predictions_lstm_tuned)

# Print the results
print("LSTM Tuned Model:")
print("Test RMSE:", test_rmse_lstm_tuned)
print("Test MSE:", test_mse_lstm_tuned)
print("Test MAPE:", test_mape_lstm_tuned)


LSTM Tuned Model:
Test RMSE: 0.019803749766050465
Test MSE: 0.0003921885047963439
Test MAPE: 3.9153156779450895


In [18]:
train_predictions_lstm_tuned = scaler.inverse_transform(train_predictions_lstm_tuned)
test_predictions_lstm_tuned = scaler.inverse_transform(test_predictions_lstm_tuned)

In [19]:
# Visualize LSTM results
train_dates = data.index[seq_length:train_size+seq_length]
test_dates = data.index[train_size+seq_length:]

fig = go.Figure()
fig.add_trace(go.Scatter(x=train_dates, y=data['Adj Close'][seq_length:train_size+seq_length], name='Actual Train Price'))
fig.add_trace(go.Scatter(x=train_dates, y=train_predictions_lstm_tuned[:, 0], name='Predicted Train Price (LSTM)'))
fig.add_trace(go.Scatter(x=test_dates, y=data['Adj Close'][train_size+seq_length:], name='Actual Test Price'))
fig.add_trace(go.Scatter(x=test_dates, y=test_predictions_lstm_tuned[:, 0], name='Predicted Test Price (LSTM)'))
fig.update_layout(title='Cryptocurrency Price Prediction (LSTM)',
                  xaxis_title='Date',
                  yaxis_title='Price')

# GBR

In [ ]:
# Load the data for XGBoost
df = pd.read_csv('/content/dataset.csv')

In [ ]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-11-09,2.053140,2.174230,1.893940,1.990770,1.990770,19192200
1,2017-11-10,2.007730,2.069470,1.644780,1.796840,1.796840,11155000
2,2017-11-11,1.786280,1.917750,1.614290,1.670470,1.670470,8178150
3,2017-11-12,1.668890,1.672800,1.462560,1.519690,1.519690,15298700
4,2017-11-13,1.526010,1.735020,1.517600,1.686620,1.686620,12238800
...,...,...,...,...,...,...,...
2015,2023-05-17,311.369812,314.269989,307.641205,314.083527,314.083527,437721579
2016,2023-05-18,314.091217,314.666290,306.649628,309.456696,309.456696,383049498
2017,2023-05-19,309.446777,311.197601,307.930054,308.964294,308.964294,334636708
2018,2023-05-20,308.982422,311.842316,308.087982,310.719971,310.719971,277393160


In [ ]:
# Data preprocessing for XGBoost
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df = df.set_index('Date')

In [ ]:
df['PriceDiff'] = df['Adj Close'].shift(-1) - df['Adj Close']

In [ ]:
df = df.dropna()

In [ ]:
# Feature selection for XGBoost
X = df[['Open', 'High', 'Low', 'Volume', 'Adj Close']]
y = df['PriceDiff']

In [ ]:
df

,Open,High,Low,Close,Adj Close,Volume,PriceDiff
Date,,,,,,,
2017-11-09,2.053140,2.174230,1.893940,1.990770,1.990770,19192200,-0.193930
2017-11-10,2.007730,2.069470,1.644780,1.796840,1.796840,11155000,-0.126370
2017-11-11,1.786280,1.917750,1.614290,1.670470,1.670470,8178150,-0.150780
2017-11-12,1.668890,1.672800,1.462560,1.519690,1.519690,15298700,0.166930
2017-11-13,1.526010,1.735020,1.517600,1.686620,1.686620,12238800,-0.094040
...,...,...,...,...,...,...,...
2023-05-16,313.542908,314.037933,309.692749,311.380035,311.380035,402988974,2.703491
2023-05-17,311.369812,314.269989,307.641205,314.083527,314.083527,437721579,-4.626831
2023-05-18,314.091217,314.666290,306.649628,309.456696,309.456696,383049498,-0.492401


In [ ]:
# Train-test split for XGBoost
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
# Hyperparameter tuning for XGBoost
params = {
    'max_depth': [3, 4, 5, 10],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 300, 500 , 1000],
    'subsample' : [0.5, 1],
}

xgb_model = xgb.XGBRegressor()
grid_search = GridSearchCV(xgb_model, params, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train_xgb, y_train_xgb)

GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'learning_rate': [0.1, 0.01],
                         'max_depth': [3, 4, 5, 10],
                         'n_estimators': [100, 300, 500, 1000],
                         'subsample': [0.5, 1]},
             scoring='neg_mean_squared_error')

In [ ]:
# Best hyperparameters for XGBoost
best_params = grid_search.best_params_
print (best_params)

{'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 100, 'subsample': 1}


In [ ]:
# Train the XGBoost model with best hyperparameters
model_xgb = xgb.XGBRegressor(**best_params)
model_xgb.fit(X_train_xgb, y_train_xgb)


In [ ]:
# Predictions using XGBoost
y_pred_xgb = model_xgb.predict(X_test_xgb)

In [ ]:
# Root Mean Squared Error (RMSE) for XGBoost
rmse_xgb = np.sqrt(mean_squared_error(y_test_xgb, y_pred_xgb))

# Calculate MSE for XGBoost
mse_xgb = mean_squared_error(y_test_xgb, y_pred_xgb)

# Calculate MAPE for XGBoost
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

mape_xgb = calculate_mape(y_test_xgb, y_pred_xgb)

# Print the results for XGBoost
print("XGBoost Model:")
print("RMSE:", rmse_xgb)
print("MSE:", mse_xgb)
print("MAPE:", mape_xgb)


XGBoost Model:
RMSE: 10.352465882494124
MSE: 7.173549848204843
MAPE: 12.590303927121685


In [ ]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df.index, y=df['Adj Close'], name='Actual Price (XGBoost)', line=dict(color='blue')))
fig2.add_trace(go.Scatter(x=df.index[-len(y_pred_xgb):], y=df['Adj Close'][-len(y_pred_xgb):] + y_pred_xgb,
                         name='Predicted Price (XGBoost)', line=dict(color='red')))
fig2.update_layout(title='Cryptocurrency Price Prediction (XGBoost)', xaxis_title='Date', yaxis_title='Price')

# Display the figures
fig2.show()

In [ ]:
# Visualization with Plotly for XGBoost
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=df.index[-len(y_pred_xgb):], y=df['Adj Close'][-len(y_pred_xgb):], name='Actual Price (XGBoost)', line=dict(color='blue')))
fig3.add_trace(go.Scatter(x=df.index[-len(y_pred_xgb):], y=df['Adj Close'][-len(y_pred_xgb):] + y_pred_xgb,
                         name='Predicted Price (XGBoost)', line=dict(color='red')))
fig3.update_layout(title='Cryptocurrency Price Prediction (XGBoost)', xaxis_title='Date', yaxis_title='Price')

# Display the figures
fig3.show()

# Komparasi

In [ ]:
# Visualize LSTM and XGBoost results
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=test_dates, y=test_predictions_lstm_tuned[:, 0], name='Predicted Test Price (LSTM)', line=dict(color='yellow')))
fig4.add_trace(go.Scatter(x=df.index[-len(y_pred_xgb):], y=df['Adj Close'][-len(y_pred_xgb):] + y_pred_xgb,
                          name='Predicted Price (XGBoost)', line=dict(color='red')))
fig4.add_trace(go.Scatter(x=df.index, y=df['Adj Close'], name='Actual Price train', line=dict(color='blue')))

fig4.update_layout(title='Cryptocurrency Price Prediction (LSTM vs XGBoost)',
                   xaxis_title='Date',
                   yaxis_title='Price')
fig4.show()


# Kesimpulan

Jadi dapat disimpulkan bahwa secara perhitungan error yang Meliputi RMSE ,MSE dan MAPE, LSTM lebih Baik daripada XGBoost Namun Secara Grafik Visual XGBoost lebih baik meski memiliki nilai error yang cukup besar
